In [1]:
import K_means_cluster_module.k_means_cluster as kmc
import global_resources as gr
import SVM.SVC as svc
import CAPM_module.get_regression_line as CAPM
import torch
import os
import numpy as np
import pandas as pd


The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\breast-cancer-wisconsin.data
Reading the input file...
Current device: Cuda.
Current training device: Cuda.

The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\breast-cancer-wisconsin.data
Reading the input file...
Creating random weights and bias with dtype: torch.float32
Current working directory: 'd:\Important Files\Repositories\Quantitative-Investment-Algorithms'.



In [2]:
cdr = gr.ch_dir_to_repo()
print(cdr)

d:\Important Files\Repositories\Quantitative-Investment-Algorithms


In [3]:
data_dir = os.path.join(cdr, 'Data', 'Stock Data for CAPM', 'Stocks')
print(data_dir)

d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data for CAPM\Stocks


In [4]:

dfs = gr.get_df_dict()



The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data for CAPM\Stocks\K线导出_002335_日线数据.xlsx
Reading the input file...

The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data for CAPM\Stocks\K线导出_002546_日线数据.xlsx
Reading the input file...

The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data for CAPM\Stocks\K线导出_002600_日线数据.xlsx
Reading the input file...

The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data for CAPM\Stocks\K线导出_002796_日线数据.xlsx
Reading the input file...

The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data for CAPM\Stocks\K线导出_300118_日线数据.xlsx
Reading the input file...

The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data for CAPM\Stocks\K线导出_30013

In [6]:
i = 0
df = dfs[list(dfs.keys())[i]]
print(df.info())
display(df.head())
na_rows = df[df.isna().any(axis=1)]
display(na_rows)
df.dropna(axis=0, inplace=True)
print(df.isna().any(axis=1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3556 entries, 0 to 3555
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   SECU_CODE   3556 non-null   object        
 1   SECU_NAME   3549 non-null   object        
 2   DATE        3549 non-null   datetime64[ns]
 3   OPENING     3549 non-null   float64       
 4   HIGHEST     3549 non-null   float64       
 5   LOWEST      3549 non-null   float64       
 6   CLOSING     3549 non-null   float64       
 7   CHANGE      3548 non-null   object        
 8   PCT_CHANGE  3548 non-null   object        
 9   VOLUME      3549 non-null   float64       
 10  AMOUNT      3549 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 305.7+ KB
None


,SECU_CODE,SECU_NAME,DATE,OPENING,HIGHEST,LOWEST,CLOSING,CHANGE,PCT_CHANGE,VOLUME,AMOUNT
0,002335,科华数据,2010-01-13,7.03,7.47,6.95,7.42,NaN,NaN,12766475.0,547214892.0
1,002335,科华数据,2010-01-14,7.35,7.96,7.33,7.56,0.1454,1.9606,8530744.0,381153458.0
2,002335,科华数据,2010-01-15,7.44,7.69,7.44,7.60,0.0422,0.5581,3734273.0,166905349.0
3,002335,科华数据,2010-01-18,7.66,7.81,7.35,7.41,-0.196,-2.5777,3078971.0,136352474.0
4,002335,科华数据,2010-01-19,7.37,7.73,7.22,7.73,0.3245,4.3805,3707511.0,163451251.0


,SECU_CODE,SECU_NAME,DATE,OPENING,HIGHEST,LOWEST,CLOSING,CHANGE,PCT_CHANGE,VOLUME,AMOUNT
0,002335,科华数据,2010-01-13,7.03,7.47,6.95,7.42,NaN,NaN,12766475.0,547214892.0
3549,nan,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3550,nan,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3551,nan,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3552,nan,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3553,nan,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3554,nan,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3555,数据来源：东方财富Choice数据,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1       False
2       False
3       False
4       False
5       False
        ...  
3544    False
3545    False
3546    False
3547    False
3548    False
Length: 3548, dtype: bool


# Use kmc to cluster the stocks into k clusters

## Getting data into k means features

### 1. Mean daily return

In [ ]:
dfs['300822'].info()

In [ ]:
import pandas as pd
import numpy as np

# Given: dfs is a dict of DataFrames keyed by security code or name, each with columns:
# ['SECU_CODE', 'SECU_NAME', 'DATE', 'OPENING', 'HIGHEST', 'LOWEST', 'CLOSING',
#  'CHANGE', 'PCT_CHANGE', 'VOLUME', 'AMOUNT']

# Prepare a container for features
features = []

for code, df in dfs.items():
    df = df.sort_values('DATE').copy()
    df.interpolate()
    # 1. Daily returns: r_t = (CLOSING_t / CLOSING_{t-1}) - 1
    df['return'] = df['CLOSING'].pct_change()

    # Basic return stats
    mu_ret = df['return'].mean()
    sigma_ret = df['return'].std()
    skew_ret = df['return'].skew()

    # 2. Maximum drawdown
    cummax = df['CLOSING'].cummax()
    drawdown = (df['CLOSING'] / cummax) - 1
    max_dd = drawdown.min()

    # 3. ATR_normalized: true range / close price
    df['prev_close'] = df['CLOSING'].shift(1)
    tr1 = df['HIGHEST'] - df['LOWEST']
    tr2 = (df['HIGHEST'] - df['prev_close']).abs()
    tr3 = (df['LOWEST'] - df['prev_close']).abs()
    df['TR'] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    # normalize ATR by closing price
    df['ATR_pct'] = df['TR'] / df['CLOSING']
    atr_pct_mean = df['ATR_pct'].mean()

    # 4. Bollinger Band width normalized: (upper - lower) / mid-band
    window = 20
    rolling_mean = df['CLOSING'].rolling(window).mean()
    rolling_std = df['CLOSING'].rolling(window).std()
    upper = rolling_mean + 2 * rolling_std
    lower = rolling_mean - 2 * rolling_std
    # absolute width
    bb_width = upper - lower
    # normalized width
    df['BB_width_pct'] = bb_width / rolling_mean
    bb_width_pct_mean = df['BB_width_pct'].mean()

    # 5. OBV normalized: cumulative signed volume / average volume
    df['sign'] = np.sign(df['return'])
    df['OBV'] = (df['sign'] * df['VOLUME']).cumsum().fillna(0)
    avg_vol = df['VOLUME'].mean()
    df['OBV_norm'] = df['OBV'] / avg_vol
    obv_norm_std = df['OBV_norm'].std()

    # 6. Volume-return correlation
    vol_ret_corr = df['VOLUME'].corr(df['return'])

    # 7. MACD normalized: (EMA12 - EMA26) / rolling mean
    ema12 = df['CLOSING'].ewm(span=12, adjust=False).mean()
    ema26 = df['CLOSING'].ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    df['MACD_pct'] = macd / rolling_mean
    macd_pct_mean = df['MACD_pct'].mean()
    macd_pct_std = df['MACD_pct'].std()

    # Collect features
    features.append({
        'SECU_CODE': code,
        'mu_return': mu_ret,
        'sigma_return': sigma_ret,
        'skew_return': skew_ret,
        'max_drawdown': max_dd,
        'ATR_pct_mean': atr_pct_mean,
        'BB_width_pct_mean': bb_width_pct_mean,
        'OBV_norm_std': obv_norm_std,
        'vol_ret_corr': vol_ret_corr,
        'MACD_pct_mean': macd_pct_mean,
        'MACD_pct_std': macd_pct_std,
    })

# Create a DataFrame of normalized features for clustering
features_df = pd.DataFrame(features)

# Return or inspect features_df
print(features_df)


In [ ]:
display(features_df.head())

In [ ]:

device = gr.set_device()
# Suppose `features_df` is your DataFrame and `device` is already defined (e.g. "cuda" or "cpu")
# 1) Select only the numeric feature columns
numeric_cols = features_df.columns.drop("SECU_CODE")

# 2) Convert to a NumPy array and then to a torch.Tensor
X = torch.tensor(
    features_df[numeric_cols].values,
    dtype=torch.float32,
    device=device
)

# feature_tensor.shape will be (n_samples, n_features)



# K means accessment

In [ ]:
variations = kmc.k_means_assessment(X, 10)
print(variations)

In [ ]:
X, best_labels, best_centroids, best_variation = kmc.WCSS_for_single_k(X = X, k = int(3))
print(best_labels, best_centroids, best_variation)